# Neural Integration and Entanglement Entropy

# imports

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.multiprocessing as mp;
%matplotlib inline
import matplotlib as mpl;
from matplotlib import pyplot as plt, font_manager as fm;
from math import gamma
import pandas as pd;
import seaborn as sns;
#"""Old level multiprocessing: complex in use and restricted to python's limits(Thread is still limited)"""
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed, wait, FIRST_COMPLETED;#new multiprocessings. Thread for multiIO, Process to multitask, as_comp... to wait
import clibs;#set of main codes
mp.set_start_method("spawn", force=True);
#os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
#extra dl model api: 일단 현재 구동 가능 후 사용해보자...
#import keras;
#from keras import layer;
#from keras import ops;

2025-03-04 11:03:27,566	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Graphic Font and fontsize Set

In [2]:
usingGffam=[f.name for f in fm.fontManager.ttflist if "Pretendard" or "Segoe" in f.name] or None;
if usingGffam and usingGffam.index("Pretendard JP Variable"):
    plt.rcParams["font.family"]="Pretendard JP Variable";
else:
    plt.rcParams["font.family"]="Segoe UI Variable Text";
plt.rc('font', size=15)        # 기본 폰트 크기
plt.rc('axes', labelsize=16)   # x,y축 label 폰트 크기
plt.rc('xtick', labelsize=14)  # x축 눈금 폰트 크기 
plt.rc('ytick', labelsize=14)  # y축 눈금 폰트 크기
plt.rc('legend', fontsize=14)  # 범례 폰트 크기
plt.rc('figure', titlesize=17) # figure title 폰트 크기
#plt.rc("figure", figsize=(5.5, 4));

# Global/Initial physical vars for learning

More detailed physical and precision-relate values are in phyp and pp

In [ ]:
modeltypes = ['Linear-Axion','Gubser-Rocha-Axion'];
#Starttime=clibs.Starttime;
#k = 1.0
mu = torch.arange(0.0, 2.0, 0.5);
beta = torch.arange(0.0, 2.0, 0.5);
#refQ = [0.0, 0.08584617289886354,0.12296275162005438,0.4714965502225489]
nords=[1, 5, 5, 5, 1];
z=torch.arange(0.0, 1.0+0.01, 0.01)
zast=z[-2];
vErr=clibs.perfparameters.vErr;

# 

# Class Init

In [ ]:
LiAxRef=clibs.refdata.Reference(modeltypes[0], z, beta, mu);
LiT, Lis=LiAxRef.sdT_sols();
clibs.dataiod.TensDFrame(False, False, z, Lis).drawseaborn("scatter", "z", "Lis");
torch.save(LiAxRef, "Liax.pt");
GRRef=clibs.refdata.Reference(modeltypes[1], z, beta, mu);
GT, Gs=GRRef.sdT_sols(z);
clibs.dataiod.TensDFrame(False, False, z, Gs).drawseaborn("scatter", "z", "Gs");
torch.save(GRRef, "GR.pt");

d:\Onedrive\OneDrive - GIST\2024-OVERSIGHT\PaperCode\2412\custompacks\refdata.py:30: SyntaxWarning: invalid escape sequence '\s'
  """re.match: 문자열의 시작부터 정규식이 일치하는지 확인합니다. re.IGNORECASE: 대소문자 구별을 무시합니다.


RuntimeError: The size of tensor a (4) must match the size of tensor b (101) at non-singleton dimension 1

In [ ]:
if  __name__=="__main__":
    RayMPWrapper=clibs.tensormultiprocessings.RayTorchMP();
    Models=[[[None for _ in enumerate(beta)] for _ in enumerate(mu)] for _ in enumerate(modeltypes)];
    Refdata=[[[None for _ in enumerate(beta)] for _ in enumerate(mu)] for _ in enumerate(modeltypes)];
    for mind, _ in enumerate(modeltypes):
        for uind, _ in enumerate(mu):
            for bind, _ in enumerate(beta):
                ModelsA[mind][uind][bind]=clibs.model.DNNP(z, nords, 1e-4);
                ModelsB[mind][uind][bind]=clibs.model.DNNP(z, nords, 1e-4, False);
    Trainer=clibs.tensormultiprocessings.raydeclare(clibs.Train(modeltypes, beta, mu, vErr));
    RayMPWrapper.multiprocess(Trainer.train(ModelsA, clibs.Pathbuild(False).rtfpath_time(res, res, pt)));
    RayMPWrapper.multiprocess(Trainer.train(ModelsB, clibs.Pathbuild(False).rtfpath_time(res, res, pt)));

# l-S polynomials

In [ ]:
class SGHDiag():
    def __init__(self, z, maxlorder, learntmodel):
        self.l0=-2.0*torch.pi*(gamma(3.0/4.0)/gamma(1.0/4.0))**2;
        self.maxlorder=maxlorder;
        self.z=z;
        self.l_m, self.S_m=learntmodel;
        
    def lfits(self):
        z=self.z;
        l_m=self.l_m;
        S_m=self.S_m;
        remainr=[[z], []];
        for lorder in range(0, self.maxlord+2, 1):
            remainr[1, lorder]=(1-(S_m/(l_m**(lorder-1))));
            s_m=(s_m-remainr[1, lorder]*(l_m**(lorder-1)));
        with ThreadPoolExecutor(max_workers=pp.cpuworkers) as io_exec:
            fullmessage=[]
            for ind in range(0, len(z), 1):
                for lorder in range(0, self.maxlorder+2, 1):
                    fullmessage.append(f"zhorizon={remainr[0, :]}, $l^{lorder-1}$={l_m**(lorder-1)}\n"+f"$c_{lorder-1}$={remainr[1, lorder, :]}\n\n");
        clibs.DataIO(remainr, "l-s series.txt").tfwrite();
        return remainr, fullmessage;
            

# Processing Part